In [1]:
## Execute once when starting cluster

# s3helper.open_bucket('mas-dse-open',region='us-west-2')
# info_files=s3helper.ls('Weather/Info')
# Without_path=[x[13:] for x in info_files]
# !mkdir /mnt/workspace/WeatherInfo
# for i in range(len(info_files)-1):   # Last entry is the directory name
#     _from=info_files[i]
#     _to='/mnt/workspace/WeatherInfo/'+Without_path[i]
#     print('From %s to %s'%(_from,_to))
#     s3helper.s3_to_local(_from,_to)
    
    
# s3helper.open_bucket('dse-weather-west-2', region="us-west-2")
# s3helper.ls('')

# s3helper.local_to_s3("/home/hadoop/s3helper.py", "fromLocal/s3helper.py")


# s3helper.s3_to_hdfs("weather.parquet", "/tmp/weather.parquet")

# s3helper.s3_to_hdfs("info/stations.parquet", "/tmp/stations.parquet")

In [2]:
# Run every time notebook restarted
from pyspark import SparkContext

sc = SparkContext()

from pyspark import SparkContext
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

df = sqlContext.sql("SELECT * FROM parquet.`/tmp/weather.parquet`")

stations=sqlContext.sql("SELECT * FROM parquet.`/tmp/stations.parquet`")

print(df.count())

df.show()

+-----------+-----------+----+--------------------+
|    Station|Measurement|Year|              Values|
+-----------+-----------+----+--------------------+
|USW00093819|       WDFM|1972|[38 58 EC 5C 38 5...|
|USW00093819|       WDFM|1973|[38 5C EC 5C 08 5...|
|USW00093819|       WDFM|1974|[EC 5C A0 51 EC 5...|
|USW00093819|       WDFM|1975|[EC 5C 38 58 38 5...|
|USW00093819|       WDFM|1976|[38 58 38 58 38 5...|
|USW00093819|       WDFM|1977|[EC 5C 38 5C A0 5...|
|USW00093819|       WDFM|1978|[38 5C 08 5B EC 5...|
|USW00093819|       WDFM|1979|[EC 5C 38 5C 08 5...|
|USW00093819|       WESD|1952|[00 7E 00 7E 00 7...|
|USW00093819|       WESD|1953|[00 7E 00 7E 00 7...|
|USW00093819|       WESD|1954|[00 7E 00 7E 00 7...|
|USW00093819|       WESD|1955|[00 7E 00 7E 00 7...|
|USW00093819|       WESD|1956|[00 7E 00 7E 00 7...|
|USW00093819|       WESD|1957|[00 7E 00 7E 00 7...|
|USW00093819|       WESD|1958|[00 7E 00 7E 00 7...|
|USW00093819|       WESD|1959|[00 7E 00 7E 00 7...|
|USW00093819

In [3]:
# Enable automiatic reload of libraries
import pandas as pd
import seaborn as sns
import os
%load_ext autoreload
%autoreload 2 # means that all modules are reloaded before every command

%pylab inline

import numpy as np

import sys
sys.path.insert(1, '/mnt/workspace/Public-DSC291/notebooks/Section2-PCA/Section2-Weather-PCA_PART2')

from lib.numpy_pack import packArray,unpackArray
#from lib.Eigen_decomp import Eigen_decomp
from lib.YearPlotter import YearPlotter
from lib.decomposer import *
from lib.Reconstruction_plots import *


from lib.import_modules import import_modules,modules
import_modules(modules)

from lib.getFiles import *

import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual,widgets
import ipywidgets as widgets

print('version of ipwidgets=',widgets.__version__)

import warnings  # Suppress Warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib
    pandas as    pd 	version=0.19.2 	required version>=0.19.2

     numpy as    np 	version=1.11.3 	required version>=1.12.0
******* Update Version ******
   sklearn as    sk 	version=0.18.1 	required version>=0.18.1

module urllib has no version
   pyspark as pyspark 	version=2.3.0 	required version>=2.1.0

ipywidgets as ipywidgets 	version=5.2.2 	required version>=6.0.0
******* Update Version ******
version of ipwidgets= 5.2.2


In [11]:
data_dir='../Data/Weather'
!ls $data_dir

CA.parquet  oni.data  STAT_CA.pickle  stations.parquet


In [8]:
# sc.stop()

In [9]:
from pyspark import SparkContext
pref = '/mnt/workspace/Public-DSC291/notebooks/Section2-PCA/Section2-Weather-PCA_PART2/'
sc = SparkContext(master="local[3]",pyFiles=[pref+'lib/numpy_pack.py',
                                             pref+'lib/spark_PCA.py',
                                             pref+'lib/computeStatistics.py',
                                             pref+'lib/Reconstruction_plots.py',
                                             pref+'lib/decomposer.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [10]:
from pickle import load
import os

state='CA'

get_weather_files(state,data_dir)
!du -h $data_dir

#read statistics
filename=data_dir+'/STAT_%s.pickle'%state
STAT,STAT_Descriptions = load(open(filename,'rb'))
measurements=STAT.keys()
print('keys from STAT=',measurements)

curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state_2/CA.tgz > ../Data/Weather/CA.tgz
tar -xzf ../Data/Weather/CA.tgz -C ../Data/Weather
rm ../Data/Weather/CA.tgz
curl https://mas-dse-open.s3.amazonaws.com/Weather/by_state_2/STAT_CA.pickle.gz > ../Data/Weather/STAT_CA.pickle.gz
gunzip ../Data/Weather/STAT_CA.pickle.gz
curl https://mas-dse-open.s3.amazonaws.com/Weather/Weather_Stations.tgz > ../Data/Weather/Weather_Stations.tgz
tar -xzf ../Data/Weather/Weather_Stations.tgz -C ../Data/Weather
rm ../Data/Weather/Weather_Stations.tgz
133M	../Data/Weather/CA.parquet
420K	../Data/Weather/stations.parquet
182M	../Data/Weather
keys from STAT= dict_keys(['TMIN', 'TMAX', 'TMAX_s20', 'SNOW', 'SNWD', 'SNWD_s20', 'SNOW_s20', 'TMIN_s20', 'PRCP', 'TOBS_s20', 'TOBS', 'PRCP_s20'])


In [ ]:
m='PRCP_s20'

Mean=STAT[m]['Mean']